<a href="https://colab.research.google.com/github/Benson0404/nsysuaop/blob/main/EX05_01_Echo_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.7 MB/s eta 0:00:00


In [4]:
# 從 Google Colab 的安全模組中匯入 userdata，用來取得儲存的憑證（如 LINE token、ngrok token）
from google.colab import userdata

# 匯入 flask-ngrok 函式，讓我們可以在 Colab 中啟動 ngrok 隧道公開 Flask 應用
from flask_ngrok import run_with_ngrok
import flask_ngrok

# ✅ 自訂函式：啟動 ngrok 隧道，並連結至本機的 port 5000
def ngrok_start():
    ngrok_set_auth_token(userdata.get('NGROK_AUTH_TOKEN'))  # 設定 ngrok 認證 token（需先在 Colab 中儲存）
    ngrok.connect(5000)  # 建立連接至 localhost:5000 的公開網址
    run_with_ngrok(app)  # 將 app 綁定至 ngrok（這行應在 app 建立之後再呼叫）

# Flask 是 Python Web Framework，這裡用來建立一個 Web 應用
from flask import Flask, request, abort

# 匯入 LINE Bot SDK 的 Webhook 處理器
from linebot.v3.webhook import WebhookHandler

# 匯入 LINE SDK 中的 Signature 錯誤類別（用來處理無效簽章的例外情況）
from linebot.v3.exceptions import InvalidSignatureError

# 匯入 LINE Messaging API 中的工具與資料模型
from linebot.v3.messaging import (
    ApiClient,                # LINE Bot API 的客戶端
    Configuration,            # 儲存存取憑證的設定物件
    MessagingApi,             # 提供訊息發送等功能
    ReplyMessageRequest,      # 回覆訊息的請求格式
    TextMessage               # 用來回傳文字訊息
)

# 匯入 webhook 的事件與內容類型
from linebot.v3.webhooks import (
    MessageEvent,             # 訊息事件
    TextMessageContent        # 純文字訊息內容
)

# 建立 Flask 應用程式物件
app = Flask(__name__)

# 建立 LINE Bot 設定：使用儲存在 userdata 裡的 Channel Access Token
configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))

# 建立 webhook handler：用來驗證和處理來自 LINE 的請求
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

# 🔁 當 LINE 的 webhook POST 訊息傳來時會進入這個 callback 路由
@app.route("/callback", methods=['POST'])
def callback():
    # 從 HTTP 標頭中取得簽名資訊
    signature = request.headers['X-Line-Signature']

    # 取得請求的主體（也就是 LINE 傳來的訊息資料）
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)  # 將收到的訊息內容輸出到伺服器 log

    # 驗證簽名並處理 webhook
    try:
        handler.handle(body, signature)  # 使用 webhook handler 驗證並處理訊息
    except InvalidSignatureError:
        # 如果簽名驗證失敗，回傳錯誤並中止處理
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'  # 驗證成功後，回傳 OK 給 LINE Server（代表 webhook 處理完畢）

# 📩 當收到文字訊息時會觸發這個事件處理函式
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    # 使用 access_token 建立 Messaging API 客戶端
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 回覆與使用者輸入相同的訊息（回聲機器人 Echo Bot）
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,  # 用來識別是哪個訊息需要被回覆
                messages=[TextMessage(text=event.message.text)]  # 回覆的訊息內容
            )
        )

# ✅ 啟動 ngrok，讓 Flask app 在公開網址上運行
ngrok_start()

# 🚀 如果這支程式是從主程式啟動，就啟動 Flask 伺服器
if __name__ == "__main__":
  app.run()

SecretNotFoundError: Secret LINE_CHANNEL_ACCESS_TOKEN does not exist.